In [1]:
!wget https://github.com/oreilly-japan/ml-security-jp/raw/master/ch02/lingspam_public.tar.gz

--2022-06-29 07:50:07--  https://github.com/oreilly-japan/ml-security-jp/raw/master/ch02/lingspam_public.tar.gz
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/oreilly-japan/ml-security-jp/master/ch02/lingspam_public.tar.gz [following]
--2022-06-29 07:50:07--  https://raw.githubusercontent.com/oreilly-japan/ml-security-jp/master/ch02/lingspam_public.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11564714 (11M) [application/octet-stream]
Saving to: ‘lingspam_public.tar.gz’

lingspam_public.tar 100%[===================>]  11.03M  --.-KB/s    in 0.1s    

2022-06-29 07:50:07 (103 MB/s) - ‘l

In [2]:
!tar zxf lingspam_public.tar.gz

In [3]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 81 kB 10.7 MB/s 
     |████████████████████████████████| 209 kB 52.7 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
     |████████████████████████████████| 49 kB 5.4 MB/s 
     |████████████████████████████████| 146 kB 59.0 MB/s 
     |████████████████████████████████| 112 kB 63.2 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=673551bffe31117bd127d311c4c1364e51670e14e333915ed7fc1011dff37aa2
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [4]:
import os
import codecs
import glob
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer


path = './lingspam_public/bare/'
text = [ ]
label = [ ]

#partディレクトリ配下のファイルを取り出す
for part in range(1,10):
  folder = os.path.join(path, 'part'+str(part))
  for filePath in glob.glob(os.path.join(folder, '*.txt')):
    if ( 'spmsga' in filePath):
      with open(filePath) as f:
        text.append(f.read())
        label.append(1)
    else :
      with open(filePath) as f:
        text.append(f.read())
        label.append(0)

#DataFrameにメールの文面とラベルを列に設定してロードする
data = pd.DataFrame()
data['Text'] = text
data['label'] = label

#TfidfVectorizerを初期化する。stop_wordsにenglishを指定し、一般的な単語を除外する
tfidf = TfidfVectorizer(stop_words="english")

X = tfidf.fit_transform(data['Text'])
colmns_names = tfidf.get_feature_names()

#Xにベクトル化した値を整形して代入
X = pd.DataFrame(X.toarray())
X = X.astype('float')
#カラム名を設定
X.columns = colmns_names
y = data['label'].astype('float')

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [5]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import optuna.integration.lightgbm as olgb
import optuna


X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, shuffle=True, random_state=101)

#LightGBM用のデータセットに変換
train = olgb.Dataset(X_train,y_train)

#パラメータの設定
params = {
    "objectives" : "binary",
    "metric" : "binary_logloss",
    "verbosity" : -1, 
    "boosting_type": "gbdt",
}

#交差検証を利用したハイパーパラメータの探索
tuner = olgb.LightGBMTunerCV(params, train, num_boost_round=100)

#ハイパーパラメータを探索
tuner.run()

[I 2022-06-29 07:51:34,394] A new study created in memory with name: no-name-af27bfad-40ed-4864-ae9f-960887103c9f
feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]

[1]	cv_agg's binary_logloss: 0.211684 + 0.005976
[2]	cv_agg's binary_logloss: 0.199816 + 0.00535674
[3]	cv_agg's binary_logloss: 0.188992 + 0.00610789
[4]	cv_agg's binary_logloss: 0.180258 + 0.0082338
[5]	cv_agg's binary_logloss: 0.173763 + 0.00901489
[6]	cv_agg's binary_logloss: 0.167174 + 0.00925015
[7]	cv_agg's binary_logloss: 0.162744 + 0.00896343
[8]	cv_agg's binary_logloss: 0.159684 + 0.010657
[9]	cv_agg's binary_logloss: 0.154571 + 0.010332
[10]	cv_agg's binary_logloss: 0.152935 + 0.0108839
[11]	cv_agg's binary_logloss: 0.149838 + 0.0114959
[12]	cv_agg's binary_logloss: 0.147493 + 0.0126043
[13]	cv_agg's binary_logloss: 0.144786 + 0.0132343
[14]	cv_agg's binary_logloss: 0.142616 + 0.0136863
[15]	cv_agg's binary_logloss: 0.141354 + 0.0134785
[16]	cv_agg's binary_logloss: 0.139618 + 0.0126813
[17]	cv_agg's binary_logloss: 0.138757 + 0.0139992
[18]	cv_agg's binary_logloss: 0.137286 + 0.0147095
[19]	cv_agg's binary_logloss: 0.135979 + 0.0159101
[20]	cv_agg's binary_logloss: 0.134882

feature_fraction, val_score: 0.222944:  14%|#4        | 1/7 [00:35<03:34, 35.81s/it]

[100]	cv_agg's binary_logloss: 0.222944 + 0.076953
[1]	cv_agg's binary_logloss: 0.210764 + 0.00469735
[2]	cv_agg's binary_logloss: 0.19975 + 0.00704605
[3]	cv_agg's binary_logloss: 0.191457 + 0.00819623
[4]	cv_agg's binary_logloss: 0.184133 + 0.00831675
[5]	cv_agg's binary_logloss: 0.176676 + 0.00845581
[6]	cv_agg's binary_logloss: 0.172566 + 0.0092841
[7]	cv_agg's binary_logloss: 0.167866 + 0.00944702
[8]	cv_agg's binary_logloss: 0.163094 + 0.00981387
[9]	cv_agg's binary_logloss: 0.157619 + 0.0105148
[10]	cv_agg's binary_logloss: 0.154183 + 0.0116386
[11]	cv_agg's binary_logloss: 0.151651 + 0.0115658
[12]	cv_agg's binary_logloss: 0.149459 + 0.0112342
[13]	cv_agg's binary_logloss: 0.147926 + 0.0115487
[14]	cv_agg's binary_logloss: 0.14607 + 0.01257
[15]	cv_agg's binary_logloss: 0.144232 + 0.0130722
[16]	cv_agg's binary_logloss: 0.143173 + 0.0136047
[17]	cv_agg's binary_logloss: 0.142209 + 0.0139989
[18]	cv_agg's binary_logloss: 0.140076 + 0.0132684
[19]	cv_agg's binary_logloss: 0.13752

feature_fraction, val_score: 0.195273:  29%|##8       | 2/7 [00:58<02:19, 27.97s/it]

...

[97]	cv_agg's binary_logloss: 0.140416 + 0.0100951
[98]	cv_agg's binary_logloss: 0.140413 + 0.0102831
[99]	cv_agg's binary_logloss: 0.140311 + 0.0101541
[100]	cv_agg's binary_logloss: 0.140215 + 0.0104063
[1]	cv_agg's binary_logloss: 0.223382 + 0.00192991
[2]	cv_agg's binary_logloss: 0.216566 + 0.0054671
[3]	cv_agg's binary_logloss: 0.211515 + 0.00599104
[4]	cv_agg's binary_logloss: 0.206204 + 0.00530978
[5]	cv_agg's binary_logloss: 0.204619 + 0.0059474
[6]	cv_agg's binary_logloss: 0.200783 + 0.00645939
[7]	cv_agg's binary_logloss: 0.196882 + 0.00781805
[8]	cv_agg's binary_logloss: 0.193789 + 0.00781991
[9]	cv_agg's binary_logloss: 0.191954 + 0.0091236
[10]	cv_agg's binary_logloss: 0.18923 + 0.0094569
[11]	cv_agg's binary_logloss: 0.186485 + 0.00989138
[12]	cv_agg's binary_logloss: 0.18421 + 0.00959803
[13]	cv_agg's binary_logloss: 0.182298 + 0.00940115
[14]	cv_agg's binary_logloss: 0.181345 + 0.0096459
[15]	cv_agg's binary_logloss: 0.180069 + 0.0101866
[16]	cv_agg's binary_logloss: 0.

min_data_in_leaf, val_score: 0.135700:  80%|########  | 4/5 [00:27<00:06,  6.95s/it]

[99]	cv_agg's binary_logloss: 0.149811 + 0.046051
[100]	cv_agg's binary_logloss: 0.149646 + 0.0462114
[1]	cv_agg's binary_logloss: 0.223382 + 0.00192991
[2]	cv_agg's binary_logloss: 0.215356 + 0.00352962
[3]	cv_agg's binary_logloss: 0.20795 + 0.00381946
[4]	cv_agg's binary_logloss: 0.205068 + 0.00466536
[5]	cv_agg's binary_logloss: 0.200114 + 0.00531509
[6]	cv_agg's binary_logloss: 0.196966 + 0.00650062
[7]	cv_agg's binary_logloss: 0.194216 + 0.00727527
[8]	cv_agg's binary_logloss: 0.191022 + 0.00804957
[9]	cv_agg's binary_logloss: 0.189111 + 0.00910736
[10]	cv_agg's binary_logloss: 0.186654 + 0.0101671
[11]	cv_agg's binary_logloss: 0.185075 + 0.0105516
[12]	cv_agg's binary_logloss: 0.18354 + 0.0103171
[13]	cv_agg's binary_logloss: 0.182015 + 0.00992079
[14]	cv_agg's binary_logloss: 0.180526 + 0.00975536
[15]	cv_agg's binary_logloss: 0.179206 + 0.0103988
[16]	cv_agg's binary_logloss: 0.178266 + 0.0108492
[17]	cv_agg's binary_logloss: 0.176983 + 0.0110505
[18]	cv_agg's binary_logloss: 0

min_data_in_leaf, val_score: 0.135700: 100%|##########| 5/5 [00:33<00:00,  6.60s/it]

[96]	cv_agg's binary_logloss: 0.141916 + 0.0120853
[97]	cv_agg's binary_logloss: 0.141779 + 0.0121519
[98]	cv_agg's binary_logloss: 0.142401 + 0.0118674
[99]	cv_agg's binary_logloss: 0.142565 + 0.0120371
[100]	cv_agg's binary_logloss: 0.142746 + 0.0124691


In [6]:
print("Best score:", 1 - tuner.best_score)
best_params = tuner.best_params

print("Best params:  ")
for key, value in best_params.items():
  print("    {}:  {}".format(key,value))

Best score: 0.8642995801841321
Best params:  
    objectives:  binary
    metric:  binary_logloss
    verbosity:  -1
    boosting_type:  gbdt
    feature_pre_filter:  False
    lambda_l1:  4.46220557300838e-06
    lambda_l2:  0.00027331319739114685
    num_leaves:  2
    feature_fraction:  0.784
    bagging_fraction:  0.8096275894349436
    bagging_freq:  6
    min_child_samples:  20


In [9]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import numpy as np

#訓練データとテストデータを設定
train_data = lgb.Dataset(X_train,y_train)
test_data = lgb.Dataset(X_test,y_test)

#ハイパーパラメータ探索で特定した値を設定
params = {
    'objective' : 'binary',
    'metric' : 'binary_logloss',
    'verbosity:' : -1,
    'boosting': 'gbdt',
    'lambda_l1' : tuner.best_params['lambda_l1'],
    'lambda_l2' : tuner.best_params['lambda_l2'],
    'num_leaves' : tuner.best_params['num_leaves'],
    'feature_fraction' : tuner.best_params['feature_fraction'],
    'bagging_fraction' : tuner.best_params['bagging_fraction'],
    'bagging_freq' : tuner.best_params['bagging_freq'],
    'min_child_samples' : tuner.best_params['min_child_samples']
}

#訓練の実施
gbm = lgb.train(
    params,
    train_data,
    num_boost_round = 100,
    verbose_eval = 0,
)

#テスト用データを使って予測する
preds = gbm.predict(X_test)
#戻り値は確率になっているので四捨五入する
pred_labels = np.rint(preds)
#正解率と混同行列の出力
print("Accuracy: {:.5f} %".format(100*accuracy_score(y_test,pred_labels)))
print(confusion_matrix(y_test,pred_labels))

Accuracy: 93.66603 %
[[478   5]
 [ 28  10]]
